In [1]:
# Feature Generation Notebook

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/all_timeline.csv')

In [3]:
df.columns

Index(['date', 'countrycode', 'countrylabel', 'showcountrylabelcases',
       'labelpositioncases', 'showcountrylabeldeaths', 'labelpositiondeaths',
       'showcountrylabelrecovered', 'labelpositionrecovered', 'casesoverride',
       'deathsoverride', 'recoveredoverride',
       'columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit',
       'emergencycasesoverride', 'emergencydeathsoverride', 'totalcases',
       'totaldeaths', 'totalrecovered'],
      dtype='object')

In [4]:
df.head(5)

,date,countrycode,countrylabel,showcountrylabelcases,labelpositioncases,showcountrylabeldeaths,labelpositiondeaths,showcountrylabelrecovered,labelpositionrecovered,casesoverride,deathsoverride,recoveredoverride,columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit,emergencycasesoverride,emergencydeathsoverride,totalcases,totaldeaths,totalrecovered
0,1/22/20,AF,Afghanistan,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/22/20,AL,Albania,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,THIS SPREADSHEET IS THE OLD VERSION. DO NOT US...,NaN,NaN,NaN,NaN,NaN
2,1/22/20,DZ,Algeria,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,TO UPDATE UK NUMBER USE THIS NEW SPREADSHEET P...,NaN,NaN,NaN,NaN,NaN
3,1/22/20,AO,Angola,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/22/20,AR,Argentina,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Convert Date Column
df.loc[:, ('date')] = df.apply(
    lambda row: pd.to_datetime(row['date'],  format='%m/%d/%y'), axis=1
)

# Sort Data by Country and Date
df = df.sort_values(by=['countrycode', 'date'])


In [6]:
# Remove Dummy Columns
del df['labelpositioncases']
del df['showcountrylabeldeaths']
del df['labelpositiondeaths']
del df['showcountrylabelrecovered']
del df['labelpositionrecovered']
del df['showcountrylabelcases']
del df['columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit']

del df['casesoverride']
del df['deathsoverride']
del df['recoveredoverride']
del df['emergencycasesoverride']
del df['emergencydeathsoverride']

In [7]:
# Remove Rows with totalcases == Nan
df = df[df['totalcases'].notna()]

# Update totaldeaths and totalrecovered columns to 0 if is Nan
df.loc[:, ('totaldeaths')] = df.apply(
    lambda row: row['totaldeaths'] if not np.isnan(row['totaldeaths']) else 0.0, axis=1
)

df.loc[:, ('totalrecovered')] = df.apply(
    lambda row: row['totalrecovered'] if not np.isnan(row['totalrecovered']) else 0.0, axis=1
)

# Commented to Generate to All Countries
#df = df[df['countrycode'] == 'BR']

df.head()

,date,countrycode,countrylabel,totalcases,totaldeaths,totalrecovered
1417,2020-01-29,AE,UAE,4.0,0.0,0.0
1596,2020-01-30,AE,UAE,4.0,0.0,0.0
1775,2020-01-31,AE,UAE,4.0,0.0,0.0
1954,2020-02-01,AE,UAE,4.0,0.0,0.0
2133,2020-02-02,AE,UAE,5.0,0.0,0.0


In [8]:
def update_column_with_previous_day_row(target_column_ix, previous_day_column_ix):
    """
    Update a Column on a Row with a Value from Previous Day Column (Basically Copy Value from Previous Day Other Column)
    
    :param target_column_ix: Index of Column to be Updated
    :param previous_day_column_ix: index of Column on Previous Day to be Coppied
    """
    
    country_code_column_ix = df.columns.get_loc('countrycode')

    for i in range(1, len(df.index)):
        
        # Check if Both Rows are the Same CountryCode
        if df.iloc[i, country_code_column_ix] == df.iloc[i-1, country_code_column_ix]: 
            df.iloc[i, target_column_ix] = df.iloc[i-1, previous_day_column_ix]
        else:
            df.iloc[i, target_column_ix] = np.nan

In [9]:
# Create all New Columns
df.loc[:, ('ft01_previous_day_totalcases')] = 0
df.loc[:, ('ft02_previous_day_totaldeaths')] = 0
df.loc[:, ('ft03_previous_day_totalrecovered')] = 0

df.loc[:, ('ft04_new_cases_per_day')] = 0
df.loc[:, ('ft05_new_deaths_per_day')] = 0
df.loc[:, ('ft06_new_recovered_per_day')] = 0

df.loc[:, ('ft07_previous_day_new_cases_per_day')] = 0
df.loc[:, ('ft08_previous_day_new_deaths_per_day')] = 0
df.loc[:, ('ft09_previous_day_new_recovered_per_day')] = 0

df.loc[:, ('ft10_cases_evolution_rate')] = 0
df.loc[:, ('ft11_deaths_evolution_rate')] = 0
df.loc[:, ('ft12_recovered_evolution_rate')] = 0

df.loc[:, ('ft13_death_percent')] = 0

In [10]:
# Export all Columns Indexes
totalcases_column_ix = df.columns.get_loc('totalcases')
totaldeaths_column_ix = df.columns.get_loc('totaldeaths')
totalrecovered_column_ix = df.columns.get_loc('totalrecovered')

ft01_previous_day_totalcases_column_ix = df.columns.get_loc('ft01_previous_day_totalcases')
ft02_previous_day_totaldeaths_column_ix = df.columns.get_loc('ft02_previous_day_totaldeaths')
ft03_previous_day_totalrecovered_column_ix = df.columns.get_loc('ft03_previous_day_totalrecovered')

ft04_new_cases_per_day_columns_ix = df.columns.get_loc('ft04_new_cases_per_day')
ft05_new_deaths_per_day_columns_ix = df.columns.get_loc('ft05_new_deaths_per_day')
ft06_new_recovered_per_day_columns_ix = df.columns.get_loc('ft06_new_recovered_per_day')

ft07_previous_day_new_cases_per_day_columns_ix = df.columns.get_loc('ft07_previous_day_new_cases_per_day')
ft08_previous_day_new_deaths_per_day_columns_ix = df.columns.get_loc('ft08_previous_day_new_deaths_per_day')
ft09_previous_day_new_recovered_per_day_columns_ix = df.columns.get_loc('ft09_previous_day_new_recovered_per_day')

ft10_cases_evolution_rate_columns_ix = df.columns.get_loc('ft10_cases_evolution_rate')
ft11_deaths_evolution_rate_columns_ix = df.columns.get_loc('ft11_deaths_evolution_rate')
ft12_recovered_evolution_rate_columns_ix = df.columns.get_loc('ft12_recovered_evolution_rate')

ft13_death_percent_columns_ix = df.columns.get_loc('ft13_death_percent')

In [11]:
# Feature 01 - Previous Day Total Cases Count
update_column_with_previous_day_row(ft01_previous_day_totalcases_column_ix, totalcases_column_ix)

# Feature 02 - Previous Day Total Deaths Count
update_column_with_previous_day_row(ft02_previous_day_totaldeaths_column_ix, totaldeaths_column_ix)

# Feature 03 - Previous Day Total Recovered Count
update_column_with_previous_day_row(ft03_previous_day_totalrecovered_column_ix, totalrecovered_column_ix)

In [12]:
# Feature 04 - New Cases per Day
df.iloc[:, ft04_new_cases_per_day_columns_ix] = \
    df.iloc[:, totalcases_column_ix] - df.iloc[:, ft01_previous_day_totalcases_column_ix]

In [13]:
# Feature 05 - New Deaths per Day
df.iloc[:, ft05_new_deaths_per_day_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] - df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

In [14]:
# Feature 06 - New Recovered per Day
df.iloc[:, ft06_new_recovered_per_day_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] - df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [15]:
# Feature 07 - Previous Day New Cases
update_column_with_previous_day_row(ft07_previous_day_new_cases_per_day_columns_ix, ft04_new_cases_per_day_columns_ix)

# Feature 08 - Previous Day New Deaths
update_column_with_previous_day_row(ft08_previous_day_new_deaths_per_day_columns_ix, ft05_new_deaths_per_day_columns_ix)

# Feature 09 - Previous Day New Recovered
update_column_with_previous_day_row(ft09_previous_day_new_recovered_per_day_columns_ix, ft06_new_recovered_per_day_columns_ix)

In [16]:
# Feature 10 - Cases Evolution Rate
df.iloc[:, ft10_cases_evolution_rate_columns_ix] = \
    df.iloc[:, totalcases_column_ix] / df.iloc[:, ft01_previous_day_totalcases_column_ix]

# Feature 11 - Deaths Evolution Rate
df.iloc[:, ft11_deaths_evolution_rate_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] / df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

# Feature 12 - Recovered Evolution Rate
df.iloc[:, ft12_recovered_evolution_rate_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] / df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [17]:
# Compute Death Percentage
df.loc[:, ('ft13_death_percent')] = df.apply(lambda row: (np.divide(row['totaldeaths'], row['totalcases']) * 100), axis=1)

In [18]:
df.head(100)

,date,countrycode,countrylabel,totalcases,totaldeaths,totalrecovered,ft01_previous_day_totalcases,ft02_previous_day_totaldeaths,ft03_previous_day_totalrecovered,ft04_new_cases_per_day,ft05_new_deaths_per_day,ft06_new_recovered_per_day,ft07_previous_day_new_cases_per_day,ft08_previous_day_new_deaths_per_day,ft09_previous_day_new_recovered_per_day,ft10_cases_evolution_rate,ft11_deaths_evolution_rate,ft12_recovered_evolution_rate,ft13_death_percent
1417,2020-01-29,AE,UAE,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,inf,NaN,NaN,0.000000
1596,2020-01-30,AE,UAE,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.00,NaN,NaN,0.000000
1775,2020-01-31,AE,UAE,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,NaN,NaN,0.000000
1954,2020-02-01,AE,UAE,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,NaN,NaN,0.000000
2133,2020-02-02,AE,UAE,5.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.25,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10741,2020-03-22,AL,Albania,76.0,2.0,2.0,76.0,2.0,2.0,0.0,0.0,0.0,6.0,0.0,2.0,1.00,1.0,1.0,2.631579
6986,2020-03-01,AM,Armenia,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
7165,2020-03-02,AM,Armenia,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.00,NaN,NaN,0.000000
7344,2020-03-03,AM,Armenia,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,NaN,NaN,0.000000


In [19]:
# DO NOT REMOVE OR CHANGE!!!!!
df.to_csv(
    'data/all_timeline_with_features.csv',
    index=False
)